# Data Set generation for detection of pilot contamination in multi user Massive MIMO Systems

Vamos fazer um esquema para gerar um dataset básico com a razão dos autovalores da matriz R, quantidade de antenas e tal pra depois tentar usar algum algoritmo de ML.

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import sys
import pandas as pd
from joblib import Parallel, delayed
from time import time
from IPython.display import display, clear_output

sys.path.append("../Python/")
import comm

### Function for signal propagation

In [2]:
def simularPropagacaoSinal(qtdSimbolosPiloto, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes=1):
    
    # SIMULANDO OS CANAIS DO USUARIO E DO ESPIAO
    Haut = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)) + 1j*np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)))
    g    = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdEspioes)) + 1j*np.random.normal(0, 1, (qtdAntenas, qtdEspioes)))

    # SEQUENCIA PILOTO ALEATORIA + MODULACAO PARA TODOS OS USUARIOS:
    bitStream = np.random.choice([0, 1], qtdSimbolosPiloto*bitsPorSimbolo*qtdUsuarios)
    symb      = comm.qpskmodulator(bitStream) # QPSK Modulator
    xp        = symb.reshape(qtdUsuarios, qtdSimbolosPiloto)

    # ESPIAO ENTRANDO NA JOGADA (SE A POTENCIA DELE FOR 0 ELE NAO ENTRA NA JOGADA):
    xpe  = np.sqrt(potenciaEspiao)*xp[0, :] # xpe vai ser a sequencia piloto do primeiro usuario multiplicada pela raiz da potencia do espiao
    xptx = np.concatenate((xp, [xpe])) # xptx sera a matriz xp com uma linha a mais: xpe
    H    = np.concatenate((Haut, g), axis=1) # H vai ser Haut com uma COLUNA a mais, que vai ser g

    # TRANSMISSAO PELO CANAL
    Y = np.dot(H, xptx) # fading
    Y = comm.awgn(Y, SNR=snr) # ruido branco
    
    return Y

### Main Function

In [3]:
def main(qtdSimbolosAtual, qtdBitsPorSimbolo, qtdAntenasAtual, qtdUsuariosAtual, potenciaEspiaoAtual, snrAtual, qtdEspioes, caminhoCSV):
    
    # PROPAGACAO DO SINAL
    Y = simularPropagacaoSinal(qtdSimbolosAtual, qtdBitsPorSimbolo, qtdAntenasAtual, qtdUsuariosAtual, potenciaEspiaoAtual, snrAtual, qtdEspioes)

    # CALCULANDO A RAZAO DOS AUTOVALORES
    razaoAtual = comm.razaoMatrizR(Y, qtdSimbolosAtual, M=qtdAntenasAtual, SNR=snrAtual)

    # FAZENDO O TARGET DO DATASET
    ataquePresente = 0
    if potenciaEspiaoAtual > 0:
        ataquePresente = 1

    # SALVANDO NO CSV - mode='a' significa append
    linhaNovaCSV = pd.DataFrame([[
        qtdUsuariosAtual, 
        qtdAntenasAtual, 
        qtdSimbolosAtual, 
        snrAtual, 
        razaoAtual, 
        potenciaEspiaoAtual, 
        ataquePresente
    ]]).to_csv(caminhoCSV, mode='a', header=False, index=False)

### System Parameters

In [4]:
# FIXED
nJobs             = 25
qtdEspioes        = 1
qtdBitsPorSimbolo = 2
repetibilidade    = 100
potenciaUsuario   = 1

# VARIABLES
rangeQtdUsuarios    = np.array([1,2,3])
rangeQtdAntenas     = np.array([64, 128, 192, 256])
rangePotenciaEspiao = np.arange(0.0, 2.51, 0.25)
rangeQtdSimbolos    = np.arange(10, 101, 5)
rangeSNRs           = np.arange(-30, 51, 2.5)

# TESTES RAPIDOS - DEIXAR COMENTADOOOOOOO
# rangeQtdUsuarios    = np.array([1]) # TESTES RAPIDOS 
# rangeQtdAntenas     = np.array([64, 128]) # TESTES RAPIDOS 
# rangePotenciaEspiao = np.arange(0.0, 1.1, 0.5) # TESTES RAPIDOS 
# rangeQtdSimbolos    = np.arange(20, 101, 20) # TESTES RAPIDOS 
# rangeSNRs           = np.arange(-10, 20, 5) # TESTES RAPIDOS 

### Simulate Scenarios While Creating CSV metadata

In [5]:
# INICIANDO O CSV
caminhoCSV = "../CSV/dataset_"+str(time())+".csv"
colunas    = [
    "qtdUsuarios", 
    "qtdAntenas", 
    "qtdSimbolos", 
    "snr", 
    "razaoAutovalores", 
    "potenciaEspiao_TARGET", 
    "ataquePresente_TARGET"
]
dataframe   = pd.DataFrame(columns=colunas).to_csv(caminhoCSV, index=False)
del dataframe
del colunas

iteracaoAtual     = 1 # para contar as iteracoes e printar um percentual de progresso
qtdPossibilidades = len(rangeQtdUsuarios) * len(rangeQtdAntenas) * len(rangePotenciaEspiao) * len(rangeQtdSimbolos) * len(rangeSNRs)

# ITERANDO TODAS AS COMBINACOES POSSIVEIS DE VARIAVEIS 
for qtdUsuariosAtual in rangeQtdUsuarios:
    for qtdAntenasAtual in rangeQtdAntenas:
        for potenciaEspiaoAtual in rangePotenciaEspiao:
            for qtdSimbolosAtual in rangeQtdSimbolos:
                for snrAtual in rangeSNRs:
                    
                    # VAMOS PARALELIZAR PQ NEM DEUS TEM A PACIENCIA NECESSARIA KKKKK
                    Parallel(n_jobs=nJobs, verbose=0)(delayed(main)(qtdSimbolosAtual, qtdBitsPorSimbolo, qtdAntenasAtual, qtdUsuariosAtual, potenciaEspiaoAtual, snrAtual, qtdEspioes, caminhoCSV) for repetibilidadeAtual in range(repetibilidade))
                    
                    # PRINT
                    clear_output(wait=True)
                    display("Progresso: " + str(100*(iteracaoAtual/qtdPossibilidades))[:7] + "%")
                    iteracaoAtual += 1

'Progresso: 100.0%'